# Preeminence tutorial

In this tutorial, you'll learn how to **create an agent** to play the game Preeminence, run games so you can track your agent's performance and step through & debug those games so you can understand what it is doing at each point.

Preeminence is a world domination game where 2-6 automated agents compete for ownership of territory on a map. Owning territories is a certified _Good Thing_ in Preeminence - it earns you extra armies, gives you access to enemy territories to expand your region of control and ultimately it can win you the game (when you own every territory on the map). Your agent plays Preeminence by **placing** armies on owned territories, then **attacking** adjacent enemy territories. Here is the _classic_ map, with territories randomly divided between 3 players (this is how the game starts):

![Example classic map](https://douglasorr.github.io/Preeminence/img/eg_classic.svg)

We'll get back to the details of the game soon, but first let's get you set up...

## 0. Setting up Preeminence

We'll use Preeminence through a web server (called _Jupyter Notebooks_) that will let you edit, run & visualise all in one place. You can run this server in Docker to keep everything tidy & predictable. To get started:

 - Install [Docker CE](https://docs.docker.com/install/)
 - Run: `$ git clone https://github.com/DouglasOrr/Preeminence.git`
   - _(clones [Preeminence](https://github.com/DouglasOrr/Preeminence))_
 - Run: `$ ./run_notebook.sh`
   - _(starts a notebook server)_
   - open the URL that is shown

In the file browser, find `Tutorial.ipynb` & open it. Now you have your own live copy of this same document. It is made up of _cells_ which can contain text or code. Start at the top & as you progress, run each cell in turn using `Shift+Enter` (you can also use `Ctrl+Enter` to repeatedly run the same cell). The remainder of this tutorial will assume you are following this approach (try it on the cell below this)...

In [ ]:
import datetime
print("Hi! I'm up-to-date as of:", datetime.datetime.now())

## Creating an agent

Now to the heart of the matter - let's create an Agent capable of playing a game! Your agent must inherit from `preem.Agent` (from now on we'll use `import preem as P` to make this `P.Agent`), and override 4 required methods. These make decisions about **army placement** (`place`, `reinforce`), strategic use of **bonus armies from redeeming cards** (`redeem`) and **army movement and attacking** (`act`).

Each method is called in a preditable order during a game:

![Agent method flow](https://douglasorr.github.io/Preeminence/img/agent_flow.svg)

 - First, the game sets up the map, assigns starting territories randomly to each player (the agent isn't involved yet).
 - Game calls `place` repeatedly (for each player in turn) to place a single army on an owned territory, until enough initial armies have been placed.
 - For each player's turn, game:
   - calls `redeem` to decide which, if any, cards should be declared for bonus armies,
   - calls `reinforce` to place multiple armies on one or more owned territories,
   - calls `act` repeatedly, until it returns a turn-ending action (such as a move).

So with that general picture in view, here is a our first agent, which implements all of these methods _correctly_ (although, not very _intelligently_):

In [ ]:
import preem as P
import random

class RandomAgent(P.Agent):
    def place(self, state):
        return random.choice(state.my_territories)

    def reinforce(self, state, count):
        return {random.choice(state.my_territories): count}

    def redeem(self, state):
        if 5 <= len(state.cards):
            sets = list(P.get_matching_sets(state.cards))
            return random.choice(sets)

    def act(self, state, earned_card):
        possible_attacks = [a for a in P.get_all_possible_attacks(state) if a.count >= 2]
        if possible_attacks:
            return random.choices(possible_attacks, weights=[a.count for a in possible_attacks])[0]
        possible_moves = P.get_all_possible_moves(state)
        if possible_moves:
            return random.choices(possible_moves, weights=[a.count for a in possible_moves])[0]
        return None

Phew! That's quite a mouthful, so let's break it down, bit by bit.
```python
class RandomAgent(P.Agent):
```
Hopefully no surprises here! (See also [Agent docs](https://douglasorr.github.io/Preeminence/#preem.Agent).) But for now we'll proceed method-by-method...

### `place`

```python
def place(self, state):
    return random.choice(state.my_territories)
```

**`place(state)`** is called repeatedly at the beginning of the game to choose which of a player's randomly assigned territories to reinforce with the initial allocation of armies. Like all `Agent` methods, it accepts a `state` object (see [PlayerState docs](https://douglasorr.github.io/Preeminence/#preem.PlayerState)), which is your agent's conduit to all info about the current state of the game. Since it's so important, let's have a look at `state`. The best way to do this is to play a game up to the point our agent has just returned from `place`:

In [ ]:
random.seed(30)
game = P.Game.start(P.Map.load('maps/quad.json'), [RandomAgent()] * 3)
event = game.next_event(player_index=0, method='place')
state = event.state

print('There are {} territories, with IDs {}'.format(
    state.map.n_territories,
    list(range(state.map.n_territories))))
print('My owned territories are', state.my_territories)
print('RandomAgent placed an army on', event.result)

Now we can see what's going on here - territories are referred to by integer ID, and the job of `place` is to choose one of the territories owned by the current player to reinforce (with a single army). It is an error to return anything except the ID of an owned territory.

> **Strategy point:** consider the pros & cons of distributing your armies - do you want to concentrate your armies on one area of the map (aggressive), or spread them out to defend all of your territories (defensive), or maybe something in between (balanced)?

### `reinforce`

```python
def reinforce(self, state, count):
    return {random.choice(state.my_territories): count}
```

**`reinforce(state, count)`** is called each turn to place reinforcement armies on owned territories. It is very similar to `place`. The difference is that reinforce places multiple armies (at least 3) in one go (the number of armies depends on the territories you control and bonuses from cards), so returns a dictionary (of `{territory: count}`) rather than a single ID:

In [ ]:
event = game.next_event(player_index=0, method='reinforce')
print('RandomAgent reinforced', event.result)

map = event.state.map
print('There are {} continents, with reinforcement values {}'.format(
    map.n_continents, map.continent_values))
print('territory-continent assignments', map.continents)

Your agent may reinforce the same territory with `count` armies, or multiple territories (totalling `count` armies deployed). It is an error to reinforce with too many or too few armies, or to try to reinforce an enemy territory (obviously!)

_Note that the game is effectively paused at the moment, and if you re-run the above cell you might get a different result each time (until the game is over and you get an error). If this happens, you can go back and re-run the previous cell (that creates `game = P.Game.start(...)`) to start a new game._

**How many armies?** The number of armies you are given to allocate in `reinforce` each turn is calculated as `territory_armies + continent_armies + redeem_set_armies`. `territory_armies` is based on the number of territories you control (see [P.count_reinforcements()](https://douglasorr.github.io/Preeminence/#preem.count_reinforcements)), and is always at least `3`. `continent_armies` are awarded when you own every territory in a group (see the groups of territories indicated by `map.continents` above) - in the above example, owning territories `[0, 1, 2, 3]` would mean you control continent `0`, therefore receive a bonus `2` armies. Finally, `redeem_set_armies` are awarded when you redeem sets of cards (see `redeem`, below).

> **Strategy point:** the value of reinforcing a territory depends on what it is connected to - which areas of the map are strategically useful for you to attack or defend? Is there any point in reinforcing a territory that cannot attack or be directly attacked?

### `redeem`

```python
def redeem(self, state):
    if 5 <= len(state.cards):
        sets = list(P.get_matching_sets(state.cards))
        return random.choice(sets)
```

**`redeem(state)`** is called each turn to interact with the game's _cards_ system. Cards are collected once a turn if your agent has conquered at least one territory, or claimed from another agent if you were the one to finish them off. They each contain one of three symbols, and can be matched into sets (but you can just use [P.get_matching_sets()](https://douglasorr.github.io/Preeminence/#preem.get_matching_sets) to do all of that work for you - see the docs if interested.) You don't have to do anything special to collect cards, however your agent implements `redeem` to decide when to trade them in for armies.

In [ ]:
event = game.next_event(player_index=0, method='redeem')
print('RandomAgent redeemed', event.result)
print('RandomAgent owns', event.state.cards)
print('value of next set:', event.state.world.next_set_value)

Your agent may choose not to redeem any cards, unless it owns 5 or more cards (see `state.cards`), in which case there must be a set, and it must return a set from `redeem` (otherwise it is an error). You can only redeem a single set at a time.

> **Strategy point:** during the game, the value of each set redeemed increases rapidly (see [P.value_of_set()](https://douglasorr.github.io/Preeminence/#preem.value_of_set)). Do you want to wait until the last possible moment to declare a set, so that it might be worth more, or is it more useful to get the armies sooner? How aggressively should I try to knock out another player, so that I can claim their cards? Does redeeming my cards early make me less of a target?

### `act`

```python
def act(self, state, earned_card):
    possible_attacks = [a for a in P.get_all_possible_attacks(state) if a.count >= 2]
    if possible_attacks:
        return random.choices(possible_attacks, weights=[a.count for a in possible_attacks])[0]
    possible_moves = P.get_all_possible_moves(state)
    if possible_moves:
        return random.choices(possible_moves, weights=[a.count for a in possible_moves])[0]
    return None
```
        
**`act(state, earned_card)`** is probably the most important method for the agent. Here, your agent may attack adjacent enemy territories multiple times, then optionally make a end-of-turn move between two adjacent owned territories. `act` can return an `Attack` (which will continue the turn after the attack has taken place), `Move` or `None` (both of which end the turn). Let's look at an attack:

In [ ]:
event = game.next_event(player_index=0, method='act', predicate=lambda e: isinstance(e.result, P.Attack))
action = event.result
print(action)
world = event.state.world
print('  owner of "from_":', world.owners[action.from_])
print('  owner of "to":', world.owners[action.to])
print('  connected?', action.to in world.map.edges[action.from_])
print('  attacker {} vs defender {}'.format(action.count, world.armies[action.to]))

Our RandomAgent uses the helpers [P.get_all_possible_attacks()](https://douglasorr.github.io/Preeminence/#preem.get_all_possible_attacks) and [P.get_all_possible_moves()](https://douglasorr.github.io/Preeminence/#preem.get_all_possible_moves) to list all the possibilities. However, as you can see from the example above, all the information needed find allowed attacks is available through `state` (either in `state.world` or `state.world.map`) - you might find this more flexible.

Attacks must be from an owned to an adjacent enemy territory, and must have a maximum count less than the number of armies on that territory (as you are not allowed to leave a territory empty). When you attack, you roll `min(3, count)` dice, and the defender rolls `min(2, armies[to])` dice. The dice are paired from highest to lowest, and ties are broken in favour of the defender. Each winning dice conquers an army from the opponent. For example, when the attacker has 3 armies and the defender has 2, the possible outcomes are 1) defender loses two armies (if there are no defending armies left, the attacker moves in with `count` and now owns the territory); 2) each lose one army (attacker gets another chance to `act`, and is allowed to try again); 3) attacker loses two armies (likewise, they will get a chance to try again via `act` as long as they have enough armies remaining on `from_`). See [P.ATTACKING_ODDS](https://douglasorr.github.io/Preeminence/#preem.ATTACKING_ODDS) for details.

As long as at least one attack in a turn has succeeded in conquering a territory, `earned_card is True` and after the turn the player is awarded an extra card from the deck. If your agent conquers the last territory belonging to a player, all of that players cards will be automatically transferred to your `state.cards`, and may be redeemed on the next turn as usual.

Moves are very similar, but only a single move is possible per turn (at the end), and the destination must be an adjacent owned territory.

> **Strategy point:** Early on the game, how desparate are you for cards? Will you fight against the odds (e.g. 1v1 or even 1v2, which favour the defender)? Later, we need to take care to make sure our armies are usable for attacks. If surrounded by owned territories, your armies are _landlocked_, and all they can do is slowly move towards the enemy at one _move_ per turn. However, on the frontier, they can _attack_ and conquer multiple territories in a single turn.

## Playing a full game

Now that we've defined, internalised & comprehended our agent, let's play a full game:

In [ ]:
random.seed(None)
P.Game.play(P.Map.load('maps/mini.json'), [RandomAgent()] * 3)

Seems self-explanatory, and is useful if we're just interested in the final outcome, but maybe we still want some logging details without stepping through manually...

In [ ]:
random.seed(21)
game = P.Game.start(P.Map.load('maps/mini.json'), agents=[RandomAgent()] * 2)
for event in game:
    print('player_{}'.format(event.state.player_index), event.method, event.args, '->', event.result)
print(game.result)

Here we've used the fact that `Game` is _iterable_ to view the whole sequence of events (rather than using `next_event()` which searches for the next matching event). Also note that `player_index` (also used to indicate the winner) is simply an index into the original `agents` array.

_Note that, as our first 1v1 game, an extra agent has appeared (called `Neutral`). The Neutral agent isn't too special - they just place their armies randomly at the beginning, then never reinforce, redeem or act - just sitting there waiting for you to attack!)_

> **Strategy point:** is attacking `Neutral` any less valuable than attacking your opponent? You can still gain cards, territories and continents, but you're not making your enemy weaker at the same time.

## Over to you...

Why not try overriding `act` or `reinforce` from `RandomAgent`? Can you make it better (or worse)? Adapt the subclass below & re-evaluate.

> E.g. one simple tweak would be to only reinforce territories that are adjacent to enemy territories (but note that it might not help win the specific game below).

In [ ]:
class NewRandomAgent(RandomAgent):
    pass  # TODO

random.seed(21)
game = P.Game.start(P.Map.load('maps/mini.json'), [NewRandomAgent(), RandomAgent()])
for event in game:
    print(type(event.agent).__name__, event.method, event.args, '->', event.result)
print(game.result)

## More debugging tools - looking at the world

Whether stepping through, or analysing complete games, these text logs can be a bit cumbersome, so let's look at them visually (you're going to be cross that I didn't you this already, but I had my reasons...)

In [ ]:
random.seed(21)
game = P.Game.start(P.Map.load('maps/mini.json'), [NewRandomAgent(), RandomAgent()])

In [ ]:
next(game)  # keep re-running this cell with Ctrl+Enter & watch what happens

The same information as we saw before can be visualised in this way. On many browsers, you should be able to hover over the image to see a tooltip with more info. Note that `Event`, `PlayerState`, `World` and `Map` can all be visualised in this way (showing less information respectively).

_Hint: if you would like better agent names on the legend of your visualisations, override `__repr__(self)` in your agent class._

Additionally (although this is slow, so probably not part of your main developing loop), you can stitch together the images into a video:

In [ ]:
random.seed(21)
P.Game.watch(P.Map.load('maps/mini.json'), [NewRandomAgent(), RandomAgent()], 'tutorial_mini.mp4', dpi=144)

## The "real" map

Most of the maps we've seen so far are small, simplified versions that will serve you well for development. But the main tournament map is called _classic_, and looks like this:

In [ ]:
map_ = P.Map.load('maps/classic.json')
print(map_)
print('Continents:', list(zip(map_.continent_names, map_.continent_values)))
map_

And here's an example of a (poorly played) game played between three random agents, on the _classic_ map:

In [ ]:
import IPython.display
IPython.display.Video('https://douglasorr.github.io/Preeminence/img/eg_classic.mp4', width=800)

## Tournaments

Soon, you will hopefully soon have loads of candidate agents (or at least settings for agents) to try. Since Preeminence includes a fair amount of randomness, we **strongly recommend comparing them across multiple games**. We have some basic tournaments that you can use to get started, or of course you can define your own based on [Game.play()](https://douglasorr.github.io/Preeminence/#preem.Game.play).

Here are the built-in 1v1 and multiplayer round robin tournaments:

In [ ]:
P.Tournament.run(P.Map.load('maps/classic.json'), [NewRandomAgent(), RandomAgent(), RandomAgent(), RandomAgent()])

In [ ]:
P.Tournament.run(P.Map.load('maps/classic.json'), [NewRandomAgent(), RandomAgent(), RandomAgent(), RandomAgent()],
                 players_per_game=4, rounds=100)

## Getting serious...

Now you've seen all that you need to get going writing the most unbeatable agent - good luck! Why don't you head over to a [fresh notebook](Working.ipynb) that's a bit less cluttered - that link will take you to a convenient skeleton.

_Thanks for following through the tutorial; have fun!_